In [31]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [32]:
batch_size = 32
num_classes = 10
epochs = 50 # ACHF - Se modifica la cantidad de epocas segun lo sugerido en el trabajo
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [33]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [34]:
model = Sequential()
model.add(Conv2D(32, (5, 5), padding='same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Conv2D(64, (5, 5), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

#Adding one convolution layer
model.add(Conv2D(64, (5, 5), padding='same'))
model.add(Activation('relu'))
# model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
# ACHF - se cambia el nombre del parametro, nueva documentacion
opt = keras.optimizers.RMSprop(lr = 0.0001, decay = 1e-6)
# opt = keras.optimizers.RMSprop(lr = 0.001)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [35]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    # ACHF - Se envia como parametro generator y se agrega un steps_per_epoch con el valor sugerido
    # cuando no se especifica, error de versionamiento
    generator = datagen.flow(x_train, y_train,batch_size=batch_size)
    model.fit_generator(generator,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        steps_per_epoch = len(generator),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.
Epoch 1/50
1563/1563 [==============================] - 82s 52ms/step - loss: 1.7982 - acc: 0.3385 - val_loss: 1.5679 - val_acc: 0.4357
Epoch 2/50
1563/1563 [==============================] - 71s 45ms/step - loss: 1.4872 - acc: 0.4609 - val_loss: 1.3151 - val_acc: 0.5315
Epoch 3/50
1563/1563 [==============================] - 73s 47ms/step - loss: 1.3238 - acc: 0.5215 - val_loss: 1.2297 - val_acc: 0.5585
Epoch 4/50
1563/1563 [==============================] - 74s 47ms/step - loss: 1.2097 - acc: 0.5679 - val_loss: 1.1391 - val_acc: 0.5946
Epoch 5/50
1563/1563 [==============================] - 76s 49ms/step - loss: 1.1262 - acc: 0.5993 - val_loss: 1.0384 - val_acc: 0.6340
Epoch 6/50
1563/1563 [==============================] - 77s 49ms/step - loss: 1.0549 - acc: 0.6234 - val_loss: 0.9936 - val_acc: 0.6545
Epoch 7/50
1563/1563 [==============================] - 76s 49ms/step - loss: 0.9939 - acc: 0.6471 - val_loss: 1.0000 - val_acc: 0.6608
Epoch 8/50
15

In [ ]:
    b   